In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.trackers import setup_logging, log_and_register, save_metadata
from datetime import datetime
from pyspark.sql import SparkSession
from src.innova.config.config import etl_config

spark = SparkSession.builder.getOrCreate()

In [4]:
BUSINESS: str = "innova"
LOAD_TS = datetime.now().strftime("%Y%m%d%H%M%S")

LAYER: str = etl_config["etl"]["layer_brz"]
STEP_NAME: str = etl_config["etl"]["steps"][0]["name"]
BASE_PATH: str = etl_config["paths"]["base_path"]
LOGS_PATH: str = etl_config["paths"]["logs_path"]      
SOURCE_PATH: str = etl_config["paths"]["source_path"]  
METADATA_PATH: str = etl_config["paths"]["metadata_path"]

BASE_BRONZE: str = f"{BASE_PATH}/{LAYER}/{BUSINESS}"

logger = setup_logging(log_path=LOGS_PATH)
BASE_BRONZE

'/alegra/data_lake/bronze/innova'

In [5]:
metadata = {}
for table_key, table_conf in etl_config["tables"].items():
    source_name = table_conf["source"]   

    input_file = f"{SOURCE_PATH}/{source_name}.csv"
    df = spark.read.option("header", True).csv(input_file)

    output_path = f"{BASE_BRONZE}/{source_name}/load_date={LOAD_TS}"
        
    metadata = log_and_register(
        table=source_name,
        df=df,
        output_path=output_path,
        load_date=LOAD_TS,
        metadata=metadata,
        logger=logger
    )

save_metadata(
    step_name=STEP_NAME,
    metadata=metadata,
    metadata_path=METADATA_PATH,
    logger=logger
)